## compute model derived fields (vorticity, steric height)

This notebook can be used to compute steric height and vorticity for regional llc4320 output.

Output is stored in new netCDF files in a "derived" directory.

Requires that an Argo climatology has been downloaded and stored locally.

NOTE: Suggested changes:
- computing strain, MLD, or other parameters
- making code work with other simulation data
- turned this notebook into a script that is called by the parent notebook
- loading the reference T/S data from ERDDAP (or another gridded Argo database)

Initial commit Aug 9, 2021 by kdrushka

In [1]:
pip install numba

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import os
# import sys
# import fsspec
import numpy as np
import glob
# import re
import gsw as sw
import xarray as xr
import dask.array as dsa
import xgcm.grid
import netCDF4 as nc4
from numba import jit

In [3]:
pwd

'/home/manjaree/.ssh/oceanliner/testing'

In [4]:
cd /home/manjaree/.ssh/oceanliner/testing

/home/manjaree/.ssh/oceanliner/testing


In [5]:
ls -l


total 8024
-rw-rw-r-- 1 manjaree manjaree    6368 Jul 14 08:46 argo_local.nc
-rw-rw-r-- 1 manjaree manjaree   97687 Jul 19 09:29 compute_model_derived_fields.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 dask-worker-space/
-rw-rw-r-- 1 manjaree manjaree  406468 Jul 14 08:46 demo.ipynb
-rw-rw-r-- 1 manjaree manjaree  853703 Jul 14 08:46 download_llc4320.ipynb
-rw-rw-r-- 1 manjaree manjaree  468805 Jul 14 08:46 final_project_updated.ipynb
-rw-rw-r-- 1 manjaree manjaree 4254897 Jul 14 08:46 get_swot_simulated_data.ipynb
drwxrwxr-x 2 manjaree manjaree    4096 Jul 14 08:46 myData/
-rw-rw-r-- 1 manjaree manjaree   66882 Jul 14 08:46 osse_code_cloud.ipynb
-rw-rw-r-- 1 manjaree manjaree  504415 Jul 14 08:46 osse_code.ipynb
-rw-rw-r-- 1 manjaree manjaree  425432 Jul 14 08:46 OSSE_code_mb.ipynb
-rw-rw-r-- 1 manjaree manjaree   47313 Jul 14 08:46 osse_tools_cloud.py
-rw-rw-r-- 1 manjaree manjaree   46215 Jul 14 08:46 osse_tools.py
-rw-rw-r-- 1 manjaree manjaree   60927 Jul 14 08:46 sn

In [6]:
llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
llc4320dir 

'/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'

In [7]:
# --------------------------------------------------------------------
# USER INPUTS:
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 

# directory where model data is stored:
#llc4320dir = '/data1/adac/mitgcm/netcdf/'
llc4320dir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
regiondir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/'
#regiondir = llc4320dir + RegionName + '/'
# directory to save derived data to - create if doesn't exist
#derivedir = regiondir + 'derived/'
derivedir = '/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/derived/'
if not(os.path.isdir(derivedir)):
    os.mkdir(derivedir)

In [18]:
# load a single file to get coordinates
fg = sorted(glob.glob(regiondir + '**nc')) # all files
i=0
thisf=fg[i]
print(thisf)
ds = xr.open_dataset(thisf)
    
# mean lat/lon of domain
xav = ds.XC.isel(j=0).mean(dim='i')
yav = ds.YC.isel(i=0).mean(dim='j')
print('center of domain: ', yav.values, ',' , xav.values)

/home/manjaree/Documents/LLC4320_pre-SWOT_10_days/ACC_SMST/osse_model_input/LLC4320_pre-SWOT_ACC_SMST_20120101.nc
center of domain:  -55.282402 , 153.0


In [9]:
# for vorticity calculation, build the xgcm grid:
# see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                 'Y':{'center': 'j', 'left': 'j_g'},
                 'T':{'center': 'time'},
                 'Z':{'center': 'k'}})

In [10]:
grid

<xgcm.Grid>
X Axis (periodic, boundary=None):
  * center   i --> left
  * left     i_g --> center
Y Axis (periodic, boundary=None):
  * center   j --> left
  * left     j_g --> center
T Axis (periodic, boundary=None):
  * center   time
Z Axis (periodic, boundary=None):
  * center   k

In [11]:
# load reference file of argo data
# NOTE: could update to pull from ERDDAP or similar
#argoclimfile = '/data1/argo/argo_CLIM_3x3.nc'
argoclimfile = '/home/manjaree/.ssh/oceanliner/testing/argo_local.nc'
argods = xr.open_dataset(argoclimfile,decode_times=False) 
# reference profiles: annual average Argo T/S using nearest neighbor
Tref = argods["temp"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
Sref = argods["salt"].sel(latitude=yav,longitude=xav, method='nearest').mean(dim='time')
# SA and CT from gsw:
# see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEV, yav)
Pref.compute()
SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                       dask='parallelized', output_dtypes=[Sref.dtype])
SAref.compute()
CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                       dask='parallelized', output_dtypes=[Sref.dtype])
CTref.compute()
Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                    dask='parallelized', output_dtypes=[Sref.dtype])
Dref.compute()
print()

In [12]:
Dref

<xarray.DataArray (LEV: 27)>
array([1026.84210864, 1026.86576986, 1026.88932413, 1026.93630681,
       1026.9810488 , 1027.08184499, 1027.21545103, 1027.42689962,
       1027.62863245, 1027.78323574, 1028.032685  , 1028.39793937,
       1028.6047681 , 1029.18590338, 1029.7443514 , 1030.29343911,
       1030.81950379, 1031.34419174, 1031.85706082, 1032.36120735,
       1032.85426514, 1033.34894085, 1033.83459135, 1034.31597463,
       1034.79590148, 1035.97796239,           nan])
Coordinates:
  * LEV        (LEV) float64 0.0 5.0 10.0 20.0 ... 1.5e+03 1.75e+03 2e+03
    latitude   float64 -55.5
    longitude  float64 152.5
    i          float32 0.0
    j          float32 0.0

In [13]:
 # create datasets for variables of interest:
ss = ds.Salt
tt = ds.Theta
pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

In [14]:
%%time
# 1. compute absolute salinity and conservative temperature
sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, output_dtypes=[ss.dtype])
sa.compute()
ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, output_dtypes=[ss.dtype])
ct.compute()
dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, output_dtypes=[ss.dtype])
dd.compute()

CPU times: user 1min 9s, sys: 2.34 s, total: 1min 11s
Wall time: 1min 11s


<xarray.DataArray (time: 24, k: 88, j: 395, i: 480)>
array([[[[1026.99810471, 1026.99895428, 1026.99853964, ...,
          1026.91353114, 1026.91628012, 1026.92135968],
         [1026.99705223, 1026.99749117, 1026.99697162, ...,
          1026.91330223, 1026.91605806, 1026.92083723],
         [1026.99609331, 1026.99628904, 1026.99554935, ...,
          1026.91315091, 1026.91588582, 1026.92045442],
         ...,
         [1026.87755162, 1026.87855173, 1026.8778911 , ...,
          1026.74122345, 1026.73705124, 1026.73326929],
         [1026.86685087, 1026.87400464, 1026.87682309, ...,
          1026.72847774, 1026.72279875, 1026.71848386],
         [1026.85282269, 1026.86056252, 1026.86961327, ...,
          1026.72041987, 1026.71528321, 1026.71160152]],

        [[1027.00417098, 1027.00499685, 1027.00455082, ...,
          1026.91813688, 1026.92093395, 1026.92604663],
         [1027.00312466, 1027.00354463, 1027.00299046, ...,
          1026.91790422, 1026.92070707, 1026.92551758],
         [1027.00217048, 1027.00234634, 1027.00158063, ...,
          1026.91774991, 1026.92053293, 1026.92513623],
...
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan]],

        [[          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         ...,
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan],
         [          nan,           nan,           nan, ...,
                    nan,           nan,           nan]]]])
Coordinates:
  * i        (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * j        (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00

In [15]:
%%time
# 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, output_dtypes=[ss.dtype])
sva.compute()

CPU times: user 12.9 s, sys: 288 ms, total: 13.2 s
Wall time: 13.2 s


<xarray.DataArray (time: 24, k: 88, j: 395, i: 480)>
array([[[[1.05251985e-06, 1.05171436e-06, 1.05210749e-06, ...,
          1.13271187e-06, 1.13010510e-06, 1.12528835e-06],
         [1.05351772e-06, 1.05310155e-06, 1.05359415e-06, ...,
          1.13292894e-06, 1.13031566e-06, 1.12578376e-06],
         [1.05442689e-06, 1.05424131e-06, 1.05494262e-06, ...,
          1.13307242e-06, 1.13047899e-06, 1.12614676e-06],
         ...,
         [1.16683050e-06, 1.16588206e-06, 1.16650857e-06, ...,
          1.29613272e-06, 1.30009044e-06, 1.30367800e-06],
         [1.17697852e-06, 1.17019425e-06, 1.16752140e-06, ...,
          1.30822330e-06, 1.31361049e-06, 1.31770371e-06],
         [1.19028243e-06, 1.18294216e-06, 1.17435878e-06, ...,
          1.31586716e-06, 1.32073995e-06, 1.32423255e-06]],

        [[1.05163444e-06, 1.05085143e-06, 1.05127432e-06, ...,
          1.13321047e-06, 1.13055813e-06, 1.12571002e-06],
         [1.05262646e-06, 1.05222828e-06, 1.05275369e-06, ...,
          1.13343109e-06, 1.13077327e-06, 1.12621168e-06],
         [1.05353112e-06, 1.05336438e-06, 1.05409035e-06, ...,
          1.13357741e-06, 1.13093838e-06, 1.12657329e-06],
...
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]],

        [[           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         ...,
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan],
         [           nan,            nan,            nan, ...,
                     nan,            nan,            nan]]]])
Coordinates:
  * i        (i) float32 0.0 1.0 2.0 3.0 4.0 ... 475.0 476.0 477.0 478.0 479.0
  * j        (j) float32 0.0 1.0 2.0 3.0 4.0 ... 390.0 391.0 392.0 393.0 394.0
  * k        (k) int32 0 1 2 3 4 5 6 7 8 9 10 ... 78 79 80 81 82 83 84 85 86 87
  * time     (time) datetime64[ns] 2012-01-01 ... 2012-01-01T23:00:00

In [16]:
%%time
# 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
# - first, interpolate Dref to the model pressure levels
Drefi = Dref.interp(LEV=-ds.Z)
dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

# steric height computation (summation/integral)
# - increase the size of Drefi and dz to match the size of sva
Db = Drefi.broadcast_like(sva)
dzb = dz.broadcast_like(sva)
dum = Db * sva * dzb
sh = dum.cumsum(dim='k')

CPU times: user 3.16 s, sys: 1.03 s, total: 4.19 s
Wall time: 4.19 s


In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
#vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')


In [ ]:
vorticity

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'X', boundary='extend')

In [ ]:
vorticity

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

In [ ]:
%%time
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.htm
#vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
vorticity = grid.interp(vorticity, 'Y', boundary='extend')


In [24]:
fis = range(len(fg))
fis

range(0, 10)

In [27]:
yn = 'y'
yn

'y'

In [ ]:
def compute_derived_fields(RegionName, datadir, start_date, ndays):
    """
    Check for derived files in {datadir}/derived and compute if the files don't exist
    """
    # directory to save derived data to - create if doesn't exist
    derivedir = datadir + 'derived/'
    if not(os.path.isdir(derivedir)):
        os.mkdir(derivedir)
        
    # files to load:
    date_list = [start_date + datetime.timedelta(days=x) for x in range(ndays)]
    target_files = [f'{datadir}LLC4320_pre-SWOT_{RegionName}_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files
    
    # list of derived files:
    derived_files = [f'{derivedir}LLC4320_pre-SWOT_{RegionName}_derived-fields_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files

        
    # loop through input files, then do the following:
    # - compute steric height
    # - interpolate vector quantities (velocity and wind) to the tracer grid
    # - rotate vectoor quantities to the geophysical (east/north) grid 
    # - compute vorticity (on the transformed grid)
    fis = range(len(target_files))
    
    cnt = 0 # count
    for fi in fis:
        # input filename:
        thisf=target_files[fi]
        # output filename:
        fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
        fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
        # check if output file already exists
        if (not(os.path.isfile(fnout))):   
            print('computing derived fields for', thisf) 
            # load file:
            ds = xr.open_dataset(thisf)
            
            # -------
            # first time through the loop, load reference profile:
            # load a single file to get coordinates
            if cnt==0:
                # mean lat/lon of domain
                xav = ds.XC.isel(j=0).mean(dim='i')
                yav = ds.YC.isel(i=0).mean(dim='j')

                # for transforming U and V, and for the vorticity calculation, build the xgcm grid:
                # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
                grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                             'Y':{'center': 'j', 'left': 'j_g'},
                             'T':{'center': 'time'},
                             'Z':{'center': 'k'}})
                

                # --- load reference file of argo data
                # here we use the 3x3 annual mean Argo product on standard produced by IRPC & distributed by ERDDAP
                # https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_defb_b79c_cb17.html
                # - download the profile closest to xav,yav once (quick), use it, then delete it.
                
                # URL gets temp & salt at all levels
                argofile = f'https://apdrc.soest.hawaii.edu/erddap/griddap/hawaii_soest_625d_3b64_cc4d.nc?temp[(0000-12-15T00:00:00Z):1:(0000-12-15T00:00:00Z)][(0.0):1:(2000.0)][({yav.data}):1:({yav.data})][({xav.data}):1:({xav.data})],salt[(0000-12-15T00:00:00Z):1:(0000-12-15T00:00:00Z)][(0.0):1:(2000.0)][({yav.data}):1:({yav.data})][({xav.data}):1:({xav.data})]'
                
                # delete the argo file if it exists 
                if os.path.isfile('argo_local.nc'):
                    os.remove('argo_local.nc')
                # use requests to get the file, and write locally:
                r = requests.get(argofile)
                file = open('argo_local.nc','wb')
                file.write(r.content)
                file.close()
                # open the argo file:
                argods = xr.open_dataset('argo_local.nc',decode_times=False)
                # get rid of time coord/dim/variable, which screws up the time in ds if it's loaded
                argods = argods.squeeze().reset_coords(names = {'time'}, drop=True) 
                # reference profiles: annual average Argo T/S using nearest neighbor
                Tref = argods["temp"]
                Sref = argods["salt"]
                # SA and CT from gsw:
                # see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
                Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEV, yav)
                Pref.compute()
                SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                                        output_dtypes=[Sref.dtype])
                SAref.compute()
                CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                                        output_dtypes=[Sref.dtype])
                CTref.compute()
                Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                                      output_dtypes=[Sref.dtype])
                Dref.compute()
                
                
                cnt = cnt+1
                print()
                
            # -------
            
            # --- COMPUTE STERIC HEIGHT IN STEPS ---
            # 0. create datasets for variables of interest:
            ss = ds.Salt
            tt = ds.Theta
            pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))
            
            # 1. compute absolute salinity and conservative temperature
            sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
            sa.compute()
            ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
            ct.compute()
            dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
            dd.compute()
            # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
            sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
            sva.compute()
            # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
            # - first, interpolate Dref to the model pressure levels
            Drefi = Dref.interp(LEV=-ds.Z)
            dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

            # steric height computation (summation/integral)
            # - increase the size of Drefi and dz to match the size of sva
            Db = Drefi.broadcast_like(sva)
            dzb = dz.broadcast_like(sva)
            dum = Db * sva * dzb
            sh = dum.cumsum(dim='k') 
            # this gives sh as a 3-d variable, (where the depth dimension 
            # represents the deepest level from which the specific volume anomaly was interpolated)
            # - but in reality we just want the SH that was determined by integrating over
            # the full survey depth, which gives a 2-d output:
            sh_true = dum.sum(dim='k') 
            
            # --- COMPUTE VORTICITY using xgcm and interpolate to X, Y
            # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
            vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
            vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')
            
            

            # --- ROTATE AND TRANSFORM VECTOR QUANTITIES ---
            # interpolate U,V and oceTAUX, oceTAUY to the tracer grid
            # and rotate them to geophysical (east, north) coordinates instead of model ones:
            # 1) regrid 
            print('interpolating to tracer grid')
            U_c = grid.interp(ds.U, 'X', boundary='extend')
            V_c = grid.interp(ds.V, 'Y', boundary='extend')
            # do the same for TAUX and TAUY:
            oceTAUX_c = grid.interp(ds.oceTAUX, 'X', boundary='extend')
            oceTAUY_c = grid.interp(ds.oceTAUY, 'Y', boundary='extend')

            # 2) rotate U and V, and taux and tauy, using rotate_vector_to_EN:
            print('rotating to east/north')
            U_transformed, V_transformed = rotate_vector_to_EN(U_c, V_c, ds['AngleCS'], ds['AngleSN'])
            oceTAUX_transformed, oceTAUY_transformed = rotate_vector_to_EN(oceTAUX_c, oceTAUY_c, ds['AngleCS'], ds['AngleSN'])

            # --- save derived fields in a new file
            # - convert sh and zeta to datasets
            # NOTE can do this more efficiently in a single line w/out converting to dataset???
            dout = vorticity.to_dataset(name='vorticity')
            sh_ds = sh.to_dataset(name='steric_height')
            dout = dout.merge(sh_ds)
            sh_true_ds = sh_true.to_dataset(name='steric_height_true')
            dout = dout.merge(sh_true_ds)
            U_transformed_ds = U_transformed.to_dataset(name='U_transformed')
            V_transformed_ds = V_transformed.to_dataset(name='V_transformed')
            oceTAUX_transformed_ds = oceTAUX_transformed.to_dataset(name='oceTAUX_transformed')
            oceTAUY_transformed_ds = oceTAUY_transformed.to_dataset(name='oceTAUY_transformed')
            dout = dout.merge(U_transformed_ds).merge(V_transformed_ds)
            dout = dout.merge(oceTAUX_transformed_ds).merge(oceTAUY_transformed_ds)
            
            
            
            # add/rename the Argo reference profile variables to dout:
            tref = Tref.to_dataset(name='Tref')
            tref = tref.merge(Sref).rename({'salt': 'Sref'}).\
                rename({'LEV':'zref','latitude':'yav','longitude':'xav'})
            # - add ref profiles to dout and drop uneeded vars/coords
            dout = dout.merge(tref).drop_vars({'longitude','latitude','LEV'})
  
    
            # - add attributes for all variables
            dout.steric_height.attrs = {'long_name' : 'Steric height',
                                    'units' : 'm',
                                    'comments_1' : 'Computed by integrating the specific volume anomaly (SVA) multiplied by a reference density, where the reference density profile is calculated from temperature & salinity profiles from the APDRC 3x3deg gridded Argo climatology product (accessed through ERDDAP). The profile nearest to the center of the domain is selected, and T & S profiles are averaged over one year before computing ref density. SVA is computed from the model T & S profiles. the Gibbs Seawater Toolbox is used compute reference density and SVA. steric_height is given at all depth levels (dep): steric_height at a given depth represents steric height signal generated by the water column above that depth - so the deepest steric_height value represents total steric height (and is saved in steric_height_true'
                                       }
            dout.steric_height_true.attrs = dout.steric_height.attrs
            
            dout.vorticity.attrs = {'long_name' : 'Vertical component of the vorticity',
                                    'units' : 's-1',
                                    'comments_1' : 'computed on DXG,DYG then interpolated to X,Y'}
            
            dout.U_transformed.attrs['long_name'] = "Horizontal velocity in the eastward direction"
            dout.U_transformed.attrs['comments_1'] = "Horizontal velocity in the eastward direction at the center of the tracer cell on the native model grid."
            dout.U_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.V_transformed.attrs['long_name'] = "Horizontal velocity in the northward direction"
            dout.V_transformed.attrs['comments_1'] = "Horizontal velocity in the northward direction at the center of the tracer cell on the native model grid."
            dout.V_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.oceTAUX_transformed.attrs['long_name'] = "Ocean surface stress in the eastward direction"
            dout.oceTAUX_transformed.attrs['comments_1'] = "Ocean surface stress due to wind and sea-ice in the eastward direction centered over the the native model grid"
            dout.oceTAUX_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."

            dout.oceTAUY_transformed.attrs['long_name'] = "Ocean surface stress in the northward direction"
            dout.oceTAUY_transformed.attrs['comments_1'] = "Ocean surface stress due to wind and sea-ice in the northward direction centered over the the native model grid"
            dout.oceTAUY_transformed.attrs['comments_3'] = "Note: this has been transformed to the tracer grid and rotated to geophysical coordinates."
            
            
            
            dout.Tref.attrs = {'long_name' : f'Reference temperature profile at {yav.data}N,{xav.data}E',
                                    'units' : 'degree_C',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            dout.Sref.attrs = {'long_name' : f'Reference salinity profile at {yav.data}N,{xav.data}E',
                                    'units' : 'psu',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            
            dout.zref.attrs = {'long_name' : f'Reference depth for Tref and Sref',
                                    'units' : 'm',
                                    'comments_1' : 'From Argo 3x3 climatology produced by APDRC'}
            
            
            # - save netcdf file with derived fields
            netcdf_fill_value = nc4.default_fillvals['f4']
            dv_encoding = {}
            for dv in dout.data_vars:
                dv_encoding[dv]={'zlib':True,  # turns compression on\
                            'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                            'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                            'dtype':'float32',\
                            '_FillValue':netcdf_fill_value}
            # save to a new file
            print(' ... saving to ', fnout)
            # TROUBLESHOOTING::::: DELETE THE RETURN LINE
            #return dout, dv_encoding
            dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)

            
            
            # release & delete Argo file
            argods.close()
#             os.remove('argo_local.nc')

In [30]:
@jit(nopython=True) # Set "nopython" mode for best performance, equivalent to @njit
# loop through files, then compute steric height, vorticity, etc. on the i/j grid
# fis = range(1)
#fis = range(len(fg))

for fi in range(len(fg)):
    # --- select data ---
    thisf=fg[fi]
    
    # check if output file already exists
    fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
    fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
    
    if (os.path.isfile(fnout) & (yn.lower() == 'y')):
        yn = input(f'\n{fnout} already exists. Overwrite? (this decision will apply to all files) [y/N]')
    if (os.path.isfile(fnout) & (yn.lower() == 'n')):
        # do nothing
        1
    elif (yn.lower() == 'y') | (not(os.path.isfile(fnout))):   
        print(thisf , '(' , fi+1, 'of', len(fis), ')')  
        ds = xr.open_dataset(thisf)

        # create datasets for variables of interest:
        ss = ds.Salt
        tt = ds.Theta
        pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

        # --- compute steric height in steps ---
        # 1. compute absolute salinity and conservative temperature
        sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
        sa.compute()
        ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
        ct.compute()
        dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        dd.compute()
        # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
        sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        sva.compute()
        # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
        # - first, interpolate Dref to the model pressure levels
        Drefi = Dref.interp(LEV=-ds.Z)
        dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

        # steric height computation (summation/integral)
        # - increase the size of Drefi and dz to match the size of sva
        Db = Drefi.broadcast_like(sva)
        dzb = dz.broadcast_like(sva)
        dum = Db * sva * dzb
        sh = dum.cumsum(dim='k')

        # --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
        #vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
        #vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

        # --- save derived fields in a new file
        # - convert sh and zeta to datasets
        #dout = vorticity.to_dataset(name='vorticity')
        sh_ds = sh.to_dataset(name='steric_height')
        dout = dout.merge(sh_ds)
        # add/rename the Argo reference profile variables
        tref = Tref.to_dataset(name='Tref')
        tref = tref.merge(Sref).rename({'SALT': 'Sref'}).\
            rename({'LEV':'zref','latitude':'yav','longitude':'xav'}).\
            drop_vars({'i','j'})
        # - add ref profiles to dout and drop uneeded vars/coords
        dout = dout.merge(tref).drop_vars({'longitude','latitude','LEV','i','j'})

        # - save netcdf file with derived fields
        netcdf_fill_value = nc4.default_fillvals['f4']
        dv_encoding = {}
        for dv in dout.data_vars:
            dv_encoding[dv]={'zlib':True,  # turns compression on\
                        'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                        'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                        'dtype':'float32',\
                        '_FillValue':netcdf_fill_value}
        # save to a new file
        print(' ... saving to ', fnout)
        dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)


SyntaxError: invalid syntax (<ipython-input-30-69d7b57ebbd2>, line 6)